In [1]:
import os
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
points_df = pd.read_csv('Points.csv', sep=';')
points_dict = dict(zip(points_df.Rank, points_df.Points))

#ENTER VENUE AND DATE HERE
    
ven_pred = 'Lousa'
date_pred = '20200322'

#ENTER RIDERS NOT PARTICIPATING IN RACE HERE
np_riders = ['Martin MAES']


In [3]:
def create_prediction_df():

    
    comp_df = pd.read_csv('riderprices2020.csv', sep=';')
    comp_df_m = comp_df[(comp_df.Gender == 'M') & (comp_df.Injured == 'No') & (~comp_df.Name.isin(np_riders))].copy()
    comp_df_w = comp_df[(comp_df.Gender == 'W') & (comp_df.Injured == 'No')].copy()

    comp_dfs = [comp_df_m, comp_df_w]
    
    
    
    for df in comp_dfs:
        
        df.drop(['Price', 'Injured', 'Gender'], axis=1, inplace=True)
        df['Venue'] = ven_pred
        df['Date'] = pd.to_datetime(date_pred, format='%Y%m%d')
        df['Year'] = df.Date.dt.year

    return (comp_df_m, comp_df_w)

In [4]:
def create_results_from_UCI_df(path):

    files = os.listdir(path)
    files_xlsx = [f for f in files if f[-4:] == 'xlsx']
    filepaths = [path + '/' + file for file in files_xlsx]

    udf = pd.DataFrame()
    for f in filepaths:
        data = pd.read_excel(f)
        stripped_f = f.rstrip('.xlsx').replace('Results_UCI_M/', '').replace('Results_UCI_W/', '')
        split_f = stripped_f.split('_')
        data['Venue'] = split_f[0]
        data['Category'] = split_f[1]
        data['Date'] = split_f[2]
        data['Name'] = data['First Name'] + ' ' + data['Last Name']
        data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d')
        data.drop(['First Name', 'Last Name', 'Phase', 'Heat', 'IRM', 'Team', 'Gender', 'Result', 'Country', 'Category', 'BIB'], axis=1, inplace=True)
        udf = udf.append(data, sort=False)
        udf.dropna(subset=['Rank'], inplace=True)
        udf.Rank = udf.Rank.astype(int)
        udf = udf[(udf.Rank < 81)]
        udf.sort_values(by=['Date', 'Rank'], inplace=True)
 
        #Create points feature
        udf['Points'] = udf.Rank.map(points_dict)
        udf.Points.fillna(0, inplace=True)
    
        #Create year feature
        udf['Year'] = udf.Date.dt.year
    
    return udf


In [5]:
def create_results_from_scraped_web_data():
    scraped_df = pd.read_excel('race_dfs_output.xlsx')
    
    #change name errors
    name_dict = {'Sam HILL' : 'Samuel HILL', 'Mick HANNAH' : 'Michael HANNAH'}
    venue_dict = {'Fort-William' : 'Fortwilliam', 'Les-Gets' : 'Lesgets', 'Les-Deux-Alpes' : 'Lesdeuxalpes'}
    scraped_df.Name.replace(name_dict, inplace=True)
    scraped_df.Venue.replace(venue_dict, inplace=True)
    scraped_df['Date'] = pd.to_datetime(scraped_df['Date'], format='%Y%m%d')
    
    #Create points feature
    scraped_df['Points'] = scraped_df.Rank.map(points_dict)
    scraped_df.Points.fillna(0, inplace=True)
    scraped_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    scraped_df = scraped_df[(scraped_df.Rank < 81)]


    scraped_df.sort_values(by=['Date', 'Rank'], inplace=True)


    
    return scraped_df
    

In [6]:
def create_features(cfdf):
    #Create Moving Average over last 3 races
    cfdf['MA3 Pos'] = cfdf.groupby('Name')['Rank'].transform(lambda x: x.rolling(3, 1).mean().shift())

    #Create last race position feature
    cfdf['Last RP'] = cfdf.groupby('Name')['Rank'].transform(lambda x: x.rolling(1, 1).mean().shift())

    #Best position in the last 5 races
    cfdf['Best pos'] = cfdf.groupby('Name')['Rank'].transform(lambda x: x.rolling(5, 1).min().shift())

    #Average position current season
    cfdf['AP this year'] = cfdf.groupby(['Name', 'Year'])['Rank'].transform(lambda x: x.rolling(10, 1).mean().shift())

    #Number of races rider has participated in current season
    #cfdf['Races CS'] = cfdf.groupby(['Name', 'Year'])['Rank'].transform(lambda x: x.rolling(10, 1).count().shift())
    #cfdf['Races CS'].fillna(0, inplace=True)
    
    return cfdf


In [7]:
def findpreviouswin(compdate, compyear, name, df):    
    avgpos_pyear = df[(df.Name == name) & (df.Year == compyear-1)].Rank.mean()
    
    return pd.Series({'AP last season' : avgpos_pyear})



In [15]:
def fantasy_model(dfinp):
    #preprocessing of dataframe
    
    #train and test set to evaluate performance of model before training model on all data and using it to predict
    #future competitions
    #df_test_snowshoe = dfinp[(dfinp['Venue'] == 'Snowshoe') & (dfinp['Date'] == '2019-09-06')].copy()
    #df_train_wo_snowshoe = dfinp[(dfinp['Venue'] != 'Snowshoe') & (dfinp['Date'] != '2019-09-06') & (dfinp.Rank < 81)].copy()
    
    
    #create df to train model on all data before using it to predict upcoming competitions, excluding upcoming competition
    df_train = dfinp[(dfinp['Venue'] != ven_pred) & (dfinp['Date'] != date_pred) & (dfinp.Rank < 81)].copy()
    #df_pred is the dataframe containing the venue to be predicted
    df_pred = dfinp[(dfinp['Venue'] == ven_pred) & (dfinp['Date'] == pd.to_datetime(date_pred, format='%Y%m%d'))].copy()


    
    y = df_train.Points
    df_train.drop('Points', axis=1, inplace=True)
    df_pred.drop('Points', axis=1, inplace=True)
    #df_test_snowshoe.drop('Points', axis=1, inplace=True)
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='median')
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    numerical_cols = ['Age', 'Year', 'MA3 Pos', 'Last RP', 'Best pos', 'AP this year', 'AP last season']
    categorical_cols = ['Name', 'Venue']
    

    #Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
     ])
    
    
    # Keep selected columns only
    my_cols = categorical_cols + numerical_cols
    X_train = df_train[my_cols].copy()
    X_test = df_pred[my_cols].copy()
    #X_test = df_test_snowshoe[my_cols].copy()
    
    
    model = XGBRegressor(n_estimators=600, learning_rate=0.005, n_jobs=4, max_depth=5, objective="reg:squarederror")
    

    
    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])
    

    scores = cross_val_score(my_pipeline, X_train, y,
                              cv=10,
                              scoring='neg_mean_absolute_error')
    print('Neg MAE:', scores)
    print("NEG MAE mean:", scores.mean())
    print('Standard deviation:', scores.std())
    
    # Preprocessing of training data, fit model 
    my_pipeline.fit(X_train, y)
    
    #Predict points distribution for the race event
    test_pred = my_pipeline.predict(X_test)
    
    return pd.Series(test_pred, df_pred.Name)


In [16]:
def race_prediction_men():
    #enter names of result folders
    path1 = 'Results_UCI_M'
    rdf1 = create_results_from_UCI_df(path1)
    
    scraped_df = create_results_from_scraped_web_data()

    result_df_M = scraped_df.append(rdf1, sort=True)
        

    comp_df_m = create_prediction_df()[0]
    
    result_df_M = result_df_M.append(comp_df_m, sort=True)

    
    #create dataframe with features
    mdf = create_features(result_df_M)

    #create feature with average number points per race for the previous season
    mdf[['AP last season']] = mdf.apply(lambda row: findpreviouswin(row.Date, row.Year, row.Name, mdf), axis=1)

    mdf.reset_index(drop=True,inplace=True)

    #Create correlation plot over the dataframe with numerical features
    #f, ax = plt.subplots(figsize=(10, 8))
    #corr = mdf.corr()
    #sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
    #        square=True, ax=ax)
    
    return fantasy_model(mdf)
race_prediction_men()

Neg MAE: [-21.53495574 -18.65106927 -19.88200309 -18.3724795  -19.07697826
 -18.13124296 -18.33080267 -19.10896065 -20.63797578 -19.86113162]
NEG MAE mean: -19.358759954062258
Standard deviation: 1.0524852454882159


Name
Amaury PIERRON       107.556709
Loic BRUNI           112.596764
Troy BROSNAN         102.243080
Danny HART            99.010574
Loris VERGIER         74.247749
                        ...    
Kaos SEAGRAVE         12.911288
Francisco PARDAL      13.845120
Noel NIEDERBERGER     19.722347
Gaëtan RUFFIN         14.380709
Rupert CHAPMAN        14.586740
Length: 87, dtype: float32

In [308]:
def race_prediction_women():
    #enter names of result folders
    path1 = 'Results_UCI_W'
    rdf1 = create_results_from_UCI_df(path1)
    #path2 = 'Results_Web_M'
    #rdf2 = create_results_from_web_df(path2)
    #result_df_M = rdf2.append(rdf1, sort=True)

    
    comp_df_w = create_prediction_df()[1]

    result_df_W = rdf1.append(comp_df_w, sort=True)



    #create dataframe with features
    wdf = create_features(result_df_W)

    #create feature with average number points per race for the previous season
    wdf[['AP last season']] = wdf.apply(lambda row: findpreviouswin(row.Date, row.Year, row.Name, wdf), axis=1)

    
    return fantasy_model(wdf)
#race_prediction_women()

In [309]:
def rider_selection(gender):
    rider_price_df = pd.read_csv('riderprices2020.csv', sep=';')
    
    
    if gender == 'M':
        predictions = race_prediction_men()
    else:
        predictions = race_prediction_women()


    

    pri_df = pd.DataFrame(predictions).reset_index()
    pri_df = pri_df.rename(columns={0: "Points"})

    pri_df = pri_df.merge(rider_price_df, left_on='Name', right_on='Name', how='left')
    pri_df['Pts./k$'] = pri_df.Points / pri_df.Price*1000
    #print(pri_df.head(50))
    
    if gender == 'M':
        pri_df = pri_df.groupby('Price').nth(list(range(4))).reset_index()
    else:
        pri_df = pri_df.groupby('Price').nth(list(range(2))).reset_index()
    
    
    return pri_df[(pri_df['Pts./k$'] > 0.1)] 


In [310]:
def find_rider_combinations():
    import itertools    
    from itertools import combinations
    
    #dataframe with riders in the men category
    ridercm_df = rider_selection('M')
    
    #dataframe with riders in the women category
    ridercw_df = rider_selection('W')
    #Create all possible combinations of riders
    #mask for non participating riders
    #nonpart = (riderc_df.Name != "Martin MAES")
    #rcomb_df = pd.DataFrame.from_records(list(itertools.combinations(riderc_df[nonpart].Name, 4)), columns=['R1', 'R2', 'R3', 'R4'])
    
    
    #Create two separate dataframes for men and women. 
    #Create combinations of 4 male riders and combinations of 2 female riders.
    
    rcombm_df = pd.DataFrame.from_records(list(itertools.combinations(ridercm_df.Name, 4)), columns=['MR1', 'MR2', 'MR3', 'MR4'])

    #Dictionaries of rider & price/points
    rpdict = dict(zip(ridercm_df.Name,ridercm_df.Price))
    rptdict = dict(zip(ridercm_df.Name,ridercm_df.Points))

    rcombm_df['CPriceM'] = rcombm_df.MR1.map(rpdict) + rcombm_df.MR2.map(rpdict) + rcombm_df.MR3.map(rpdict) + rcombm_df.MR4.map(rpdict)
    rcombm_df['CPointsM'] = rcombm_df.MR1.map(rptdict) + rcombm_df.MR2.map(rptdict) + rcombm_df.MR3.map(rptdict) + rcombm_df.MR4.map(rptdict)
    rcombm_df['Comb. Pts/k$'] = rcombm_df.CPointsM / rcombm_df.CPriceM * 1000

    rcombm_df.sort_values(by=['CPointsM'], ascending = False, inplace=True)
    rcombm_df['RidersM'] = list(zip(rcombm_df.MR1, rcombm_df.MR2, rcombm_df.MR3, rcombm_df.MR4))

    #rcombm_df.drop(rcombm_df[rcombm_df.CPriceM > 50000].index, inplace=True)

    rcombm_df = rcombm_df[(rcombm_df.CPriceM < 1500000) & (rcombm_df.CPriceM > 340000)].copy()
    
    #Creation of dataframes for female riders
    rcombw_df = pd.DataFrame.from_records(list(itertools.combinations(ridercw_df.Name, 2)), columns=['WR1', 'WR2'])
    
    rpwdict = dict(zip(ridercw_df.Name,ridercw_df.Price))
    rptwdict = dict(zip(ridercw_df.Name,ridercw_df.Points))
    
    rcombw_df['CPriceW'] = rcombw_df.WR1.map(rpwdict) + rcombw_df.WR2.map(rpwdict) 
    rcombw_df['CPointsW'] = rcombw_df.WR1.map(rptwdict) + rcombw_df.WR2.map(rptwdict) 
    rcombw_df['Comb. Pts/k$ W'] = rcombw_df.CPointsW / rcombw_df.CPriceW * 1000

    rcombw_df.sort_values(by=['CPointsW'], ascending = False, inplace=True)
    rcombw_df['RidersW'] = list(zip(rcombw_df.WR1, rcombw_df.WR2))

    #find best combinations of both men and women
    combinate_df = pd.DataFrame.from_records(list(itertools.product(rcombw_df.RidersW, rcombm_df.RidersM)), columns=['RidersW', 'RidersM'])
    
    
    combinate_df = combinate_df.merge(rcombw_df, left_on='RidersW', right_on='RidersW')
    combinate_df = combinate_df.merge(rcombm_df, left_on='RidersM', right_on='RidersM')


    
    
    combinate_df['Combined Price'] = combinate_df.CPriceW + combinate_df.CPriceM
    combinate_df['Combined Points'] = combinate_df.CPointsW + combinate_df.CPointsM

    combinate_df.drop(['MR1', 'MR2', 'MR3', 'MR4', 'WR1', 'WR2', 'CPriceW', 'CPriceM', 'CPointsW', 'CPointsM'], axis=1, inplace=True)


    combinate_df.sort_values(by=['Combined Points'], ascending = False, inplace=True)
    
    return combinate_df[(combinate_df['Combined Price'] < 1500001)]



In [311]:
find_rider_combinations()

Neg MAE: [-20.20925308 -19.23591384 -18.66822861 -18.9025378  -20.46353049]
NEG MAE mean: -19.495892762504287
Standard deviation: 0.7141278316338439
Neg MAE: [-21.24704232 -20.17804755 -16.781443   -18.29845971 -25.28411114]
NEG MAE mean: -20.357820743870086
Standard deviation: 2.9025098128010756


,RidersW,RidersM,Comb. Pts/k$ W,Comb. Pts/k$,Combined Price,Combined Points
1998618,"(Morgane CHARRE, Myriam NICOLE)","(Taylor VERNON, Brendan FAIRCLOUGH, Aaron GWIN...",0.459818,0.275381,1500000,509.900717
2001018,"(Morgane CHARRE, Myriam NICOLE)","(Joe BREEDEN, Brendan FAIRCLOUGH, Aaron GWIN, ...",0.459818,0.275365,1500000,509.885302
2257218,"(Morgane CHARRE, Myriam NICOLE)","(Baptiste PIERRON, David TRUMMER, Charlie HARR...",0.459818,0.272929,1500000,507.510098
2276418,"(Morgane CHARRE, Myriam NICOLE)","(Joshua BARTH, Brendan FAIRCLOUGH, Aaron GWIN,...",0.459818,0.284427,1460000,507.343550
2319618,"(Morgane CHARRE, Myriam NICOLE)","(Brendan FAIRCLOUGH, David TRUMMER, Charlie HA...",0.459818,0.287011,1450000,506.889523
...,...,...,...,...,...,...
74867389,"(Melanie CHAPPAZ, Vali HÖLL)","(Thibault RUFFIN, Alexandre FAYOLLE, Lucas CRU...",0.284697,0.152088,750000,163.794289
74867388,"(Cecile RAVANEL, Vali HÖLL)","(Thibault RUFFIN, Alexandre FAYOLLE, Lucas CRU...",0.170818,0.152088,1000000,163.794289
74867387,"(Janine HUBSCHER, Mille JOHNSET)","(Thibault RUFFIN, Alexandre FAYOLLE, Lucas CRU...",0.667258,0.152088,535000,163.794289
74867386,"(Melanie CHAPPAZ, Cecile RAVANEL)","(Thibault RUFFIN, Alexandre FAYOLLE, Lucas CRU...",0.305032,0.152088,725000,163.794289
